In [29]:
#instalaçao das bibliotecas (opicional)
#import pip
#pip.main(["install","vscode-ipynb-py-convert"])


In [30]:
#Pandas: Possui inúmeras funções e comandos para importar arquivos, analisar dados, tratar dados, etc.
import pandas as pd # importando a biblioteca pandas

#Numpy: Possui uma série de funções e comandos para trabalharmos com números de forma em geral(formatação, calculos, etc)
import numpy as np # importando a biblioteca numpy

#Time: Possui uma série de funções e comandos para trabalharmos com tempo
import time # importando a biblioteca time

#glob: Possui uma série de funções e comandos para trabalharmos com arquivos
import glob # importando a biblioteca glob

#os: Possui uma série de funções e comandos para trabalharmos com arquivos
import os # importando a biblioteca os

#XlsxWriter: Possui uma série de funções e comandos para trabalharmos com arquivos excel
import xlsxwriter # importando a biblioteca xlsxwriter

#Pyexcel: Possui uma série de funções e comandos para trabalharmos com arquivos excel
import pyexcel as pe # importando a biblioteca pyexcel

#Warnings: Possui detalhes sobre os avisos e alertas que aparecem, porém podemos utiliza-lo também para que os alertas de
#futuras atualizações e metodos depreciados não sejam exibidos
import warnings
warnings.filterwarnings("ignore") 
tempo_inicial = time.time() # tempo inicial para calcular o tempo de execução do código

from glob import glob # Utilizado para listar arquivos de um diretório

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Colorindo o texto
class colors:
    OK = '\033[92m'
    ERRO = '\033[91m'
    ALERTA = '\033[43m'
    NORMAL = '\033[0m'

## Importação dos Dados do CNES 

In [31]:
df_cnes_leitos = pd.read_csv('BASE\.BASE_CNES_LEITOS.csv', sep=';', encoding='latin-1', dtype=str)
df_cnes_leitos.head()

,CO_UNIDADE,CO_CNES,NO_RAZAO_SOCIAL,NO_FANTASIA,TP_UNIDADE,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,CO_MOTIVO_DESAB,TP_GESTAO,CO_LEITO,CO_TIPO_LEITO,QT_EXIST,QT_SUS
0,4208307575521,7575521,CLINICA JANE REINERT CARRAMILLO,CLINICA JANE REINERT CARRAMILLO,22,42,420830,NaN,M,NaN,NaN,NaN,NaN
1,4208307575742,7575742,LE LIS CLINICA,LE LIS CLINICA,22,42,420830,NaN,M,NaN,NaN,NaN,NaN
2,4208907566735,7566735,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,UNIDADE BASICA DE SAUDE AMABILE PEDROTTI,2,42,420890,NaN,M,NaN,NaN,NaN,NaN
3,4209007574746,7574746,DORE CLINICA DE FISIOTERAPIA E POSTUROLOGIA LTDA,FISIOMAIS,36,42,420900,NaN,M,NaN,NaN,NaN,NaN
4,4209107571100,7571100,IMAGEM X RAD TOMO E DIAGNOSTICO ODONTOLOGICO LTDA,IMAGEM X RADIOLOGIA MATRIZ,36,42,420910,NaN,M,NaN,NaN,NaN,NaN


In [32]:
df_cnes_habilitacao = pd.read_csv('BASE\.BASE_CNES_HABILITACAO.csv', sep=';', encoding='latin-1', dtype=str)
df_cnes_habilitacao.head()

,CO_UNIDADE,CO_CNES,NO_RAZAO_SOCIAL,NO_FANTASIA,TP_UNIDADE,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,CO_MOTIVO_DESAB,TP_GESTAO,CO_CODIGO_GRUPO,NO_DESCRICAO_GRUPO
0,4208307575521,7575521,CLINICA JANE REINERT CARRAMILLO,CLINICA JANE REINERT CARRAMILLO,22,42,420830,NaN,M,NaN,NaN
1,4208307575742,7575742,LE LIS CLINICA,LE LIS CLINICA,22,42,420830,NaN,M,NaN,NaN
2,4208907566735,7566735,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,UNIDADE BASICA DE SAUDE AMABILE PEDROTTI,2,42,420890,NaN,M,913.0,ADESAO DO ESTABELECIMENTO AO PROJETO MAIS MÉDICOS
3,4208907566735,7566735,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,UNIDADE BASICA DE SAUDE AMABILE PEDROTTI,2,42,420890,NaN,M,7111.0,ESTABELECIMENTO DE SAUDE SEM GERACAO DE CREDIT...
4,4209007574746,7574746,DORE CLINICA DE FISIOTERAPIA E POSTUROLOGIA LTDA,FISIOMAIS,36,42,420900,NaN,M,NaN,NaN


In [33]:
df_cnes_servicos = pd.read_csv('BASE\.BASE_CNES_SERVICOS.csv', sep=';', encoding='latin-1', dtype=str)
df_cnes_servicos.head()

,CO_UNIDADE,CO_CNES,NO_RAZAO_SOCIAL,NO_FANTASIA,TP_UNIDADE,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,CO_MOTIVO_DESAB,TP_GESTAO,CO_SERVICO,CO_CLASSIFICACAO
0,4208307575521,7575521,CLINICA JANE REINERT CARRAMILLO,CLINICA JANE REINERT CARRAMILLO,22,42,420830,NaN,M,131.0,2.0
1,4208307575742,7575742,LE LIS CLINICA,LE LIS CLINICA,22,42,420830,NaN,M,NaN,NaN
2,4208907566735,7566735,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,UNIDADE BASICA DE SAUDE AMABILE PEDROTTI,2,42,420890,NaN,M,112.0,1.0
3,4208907566735,7566735,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,UNIDADE BASICA DE SAUDE AMABILE PEDROTTI,2,42,420890,NaN,M,119.0,1.0
4,4208907566735,7566735,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,UNIDADE BASICA DE SAUDE AMABILE PEDROTTI,2,42,420890,NaN,M,174.0,1.0


## Importação dos Dados do SIGTAP

In [34]:
df_sigtap = pd.read_csv('BASE\.BASE_SIGTAP_GERAL.csv', sep=';', encoding='latin-1', dtype=str)
df_sigtap['COD_PROCEDIMENTO']= df_sigtap['COD_PROCEDIMENTO'].astype(int) # Converte a coluna 'COD_PROCEDIMENTO' para string
df_sigtap['QT_PROD AIH 2018 a 2022']= df_sigtap['QT_PROD AIH 2018 a 2022'].astype(int) # Converte a coluna 'QT_PROD AIH 2018 a 2022' para string
df_sigtap.head() # Exibe as 5 primeiras linhas do arquivo

,Procedimentos envolvidos,SUBGRUPO,COD_PROCEDIMENTO,DEC_PROCEDIMENTO,Dados de Produção Brasil (2018 a 2022),% ELET SIASUS,QT_PROD AIH 2018 a 2022,% ELET AIH,EXIGE SERVIÇO,EXIGE HABILITACAO,BPA_I,AIH,APAC,CO_REGISTRO,NO_REGISTRO,CO_SERVICO,CO_CLASSIFICACAO,CO_HABILITACAO
0,0401,"PEQUENAS CIRURGIAS E CIRURGIAS DE PELE, TECIDO...",401020010,ENXERTO COMPOSTO,0,0,2972,0.5259084791386271,-,-,-,SIM,-,3,AIH (Proc. Principal),NaN,NaN,NaN
1,0401,"PEQUENAS CIRURGIAS E CIRURGIAS DE PELE, TECIDO...",401020029,ENXERTO DERMO-EPIDERMICO,0,0,23406,0.1587627104161326,-,-,-,SIM,-,3,AIH (Proc. Principal),NaN,NaN,NaN
2,0401,"PEQUENAS CIRURGIAS E CIRURGIAS DE PELE, TECIDO...",401020037,ENXERTO LIVRE DE PELE TOTAL,0,0,13725,0.2255009107468124,-,-,-,SIM,-,3,AIH (Proc. Principal),NaN,NaN,NaN
3,0401,"PEQUENAS CIRURGIAS E CIRURGIAS DE PELE, TECIDO...",401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",0,0,56006,0.7712745063028961,-,-,-,SIM,-,3,AIH (Proc. Principal),NaN,NaN,NaN
4,0401,"PEQUENAS CIRURGIAS E CIRURGIAS DE PELE, TECIDO...",401020053,EXCISAO E SUTURA DE LESAO NA PELE C/ PLASTICA ...,0,0,149988,0.7915166546657066,-,-,-,SIM,-,3,AIH (Proc. Principal),NaN,NaN,NaN


## Importação TETO MAC

In [35]:
df_teto = pd.read_csv('BASE\.BASE_TETO_MAC.csv', sep=';', encoding='latin-1', dtype=str)
df_teto.head()

,Sigla UF,Código IBGE,Estado / Município,Código Gestão,Descrição Gestão,Teto Financeiro MAC - Valores Anuais (R$)
0,AC,120000,ACRE,2,Gestão Estadual,204774437.91
1,AC,120001,ACRELÂNDIA,1,Gestão Municipal,372836.91
2,AC,120005,ASSIS BRASIL,1,Gestão Municipal,0.0
3,AC,120010,BRASILÉIA,1,Gestão Municipal,339660.0
4,AC,120013,BUJARI,1,Gestão Municipal,0.0


## Importação da PLANILHA  

### Analise e tratamento da PLANILHA ABA 1 

In [36]:
df_planilha = glob('PLANILHA\*.xlsx')[0] # Planilha para ser validada

In [37]:
df_planilha_aba1 = pd.read_excel(df_planilha, sheet_name='Ident. Fila na UF') # Lê o arquivo excel
df_planilha_aba1.head() # Exibe as 5 primeiras linhas do arquivo

,PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Planilha protegida devido as fórmulas existent...,NaN,NaN,NaN,NaN,NaN,bahia,VRS 4
1,CÓDIGO DO PROCEDIMENTO NO SIGTAP,PROCEDIMENTO CIRÚRGICO,QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/...,Redução do Tamanho da Fila (%),Prazo para reduzir o % proposto (em meses),Qtde de cirurgias a serem feitas no prazo pact...,SQ (CODIGO Interno,NaN
2,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",166,1,9,166,100001,NaN
3,401020053,EXCISAO E SUTURA DE LESAO NA PELE C/ PLASTICA ...,50,1,9,50,100002,NaN
4,401020088,EXERESE DE CISTO SACRO-COCCIGEO,594,0.7,9,415.8,100003,NaN


In [38]:
df_planilha_aba1.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA':'COD_PROCEDIMENTO','Unnamed: 1':'DESC_PROCEDIMENTO','Unnamed: 2':'QUANT_FILA', 
                                 'Unnamed: 3':'PERC_REDUCAO', 'Unnamed: 4':'TEMPO_MESES', 'Unnamed: 5':'QUANT_REDUCAO', 'Unnamed: 6':'LINHA'}, inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'
df_planilha_aba1.drop(0, inplace=True) # Remove a primeira linha do arquivo
df_planilha_aba1.drop(1, inplace=True) # Remove a segunda linha do arquivo
df_planilha_aba1.drop(df_planilha_aba1.tail(1).index,inplace=True) # Removendo a última linha do arquivo
df_planilha_aba1.dropna(subset=['COD_PROCEDIMENTO'], inplace=True) # Removendo linhas com valores nulos
df_planilha_aba1.drop('Unnamed: 7', axis=1, inplace=True) # Removendo coluna 'Unnamed: 7'
df_planilha_aba1['COD_PROCEDIMENTO'] = df_planilha_aba1['COD_PROCEDIMENTO'].astype(int) # Converte a coluna 'COD_PROCEDIMENTO' para string
df_planilha_aba1['PERC_REDUCAO'] = df_planilha_aba1['PERC_REDUCAO'].astype(float) # Converte a coluna 'QUANT_FILA' para string
df_planilha_aba1['QUANT_REDUCAO'] = df_planilha_aba1['QUANT_REDUCAO'].astype(int) # Converte a coluna 'QUANT_REDUCAO' para string
df_planilha_aba1['QUANT_FILA'] = df_planilha_aba1['QUANT_FILA'].astype(int) # Converte a coluna 'QUANT_FILA' para string

df_planilha_aba1.head() # Exibe as 5 primeiras linhas do arquivo

,COD_PROCEDIMENTO,DESC_PROCEDIMENTO,QUANT_FILA,PERC_REDUCAO,TEMPO_MESES,QUANT_REDUCAO,LINHA
2,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",166,1.0,9,166,100001
3,401020053,EXCISAO E SUTURA DE LESAO NA PELE C/ PLASTICA ...,50,1.0,9,50,100002
4,401020088,EXERESE DE CISTO SACRO-COCCIGEO,594,0.7,9,415,100003
5,402010035,TIREOIDECTOMIA PARCIAL,22,0.0,9,0,100004
6,402010043,TIREOIDECTOMIA TOTAL,279,0.9,9,251,100005


### Verificar procedimento duplicado

In [39]:
# Verifica se há valores duplicados na coluna "COD_PROCEDIMENTO"
df_planilha_aba1['DUPLICADO'] = df_planilha_aba1['COD_PROCEDIMENTO'].duplicated(keep=False)

# Cria uma nova coluna com as mensagens correspondentes
df_planilha_aba1['M_DUPLICADO'] = np.where(df_planilha_aba1['DUPLICADO'], 'SIM', '-')
df_planilha_aba1.drop('DUPLICADO', axis=1, inplace=True) # Remove a coluna 'DUPLICADO'
# Exibe as 5 primeiras linhas do arquivo com a nova coluna
df_planilha_aba1.head()

,COD_PROCEDIMENTO,DESC_PROCEDIMENTO,QUANT_FILA,PERC_REDUCAO,TEMPO_MESES,QUANT_REDUCAO,LINHA,M_DUPLICADO
2,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",166,1.0,9,166,100001,-
3,401020053,EXCISAO E SUTURA DE LESAO NA PELE C/ PLASTICA ...,50,1.0,9,50,100002,-
4,401020088,EXERESE DE CISTO SACRO-COCCIGEO,594,0.7,9,415,100003,-
5,402010035,TIREOIDECTOMIA PARCIAL,22,0.0,9,0,100004,-
6,402010043,TIREOIDECTOMIA TOTAL,279,0.9,9,251,100005,-


### Verificar se existe atendimento na Quantidade de Reduçao numero inferior a 1 procedimento. 

In [40]:
# Adicionando nova coluna 'QUANT_ZERO' com a mensagem correspondente

def mensagem(quant_zero):
    if quant_zero ['QUANT_REDUCAO'] < 1 and quant_zero['PERC_REDUCAO'] > 0 :
        return 'ERRO_QUANT'
    else:
        return '-'
df_planilha_aba1['QUANT_ZERO'] = df_planilha_aba1.apply(mensagem, axis=1)

# Exibindo as primeiras linhas do dataframe com a nova coluna
df_planilha_aba1.head() 


,COD_PROCEDIMENTO,DESC_PROCEDIMENTO,QUANT_FILA,PERC_REDUCAO,TEMPO_MESES,QUANT_REDUCAO,LINHA,M_DUPLICADO,QUANT_ZERO
2,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",166,1.0,9,166,100001,-,-
3,401020053,EXCISAO E SUTURA DE LESAO NA PELE C/ PLASTICA ...,50,1.0,9,50,100002,-,-
4,401020088,EXERESE DE CISTO SACRO-COCCIGEO,594,0.7,9,415,100003,-,-
5,402010035,TIREOIDECTOMIA PARCIAL,22,0.0,9,0,100004,-,-
6,402010043,TIREOIDECTOMIA TOTAL,279,0.9,9,251,100005,-,-


#### Procedimento requer habilitação

In [41]:
df_sigtap_h = df_sigtap[['COD_PROCEDIMENTO','EXIGE HABILITACAO','CO_HABILITACAO']] # Cria um novo dataframe com as colunas 'COD_PROCEDIMENTO','EXIGE HABILITACAO','CO_HABILITACAO'
df_sigtap_h.drop_duplicates(subset='COD_PROCEDIMENTO', keep='first', inplace=True) # Remove os valores duplicados da coluna 'COD_PROCEDIMENTO'
df_planilha_aba1['PROC_HABILITACAO'] = df_planilha_aba1['COD_PROCEDIMENTO'].map(df_sigtap_h.set_index('COD_PROCEDIMENTO')['EXIGE HABILITACAO']) # Adiciona uma nova coluna com a informação de habilitação do procedimento
df_planilha_aba1.head() # Exibe as 5 primeiras linhas do arquivo

,COD_PROCEDIMENTO,DESC_PROCEDIMENTO,QUANT_FILA,PERC_REDUCAO,TEMPO_MESES,QUANT_REDUCAO,LINHA,M_DUPLICADO,QUANT_ZERO,PROC_HABILITACAO
2,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",166,1.0,9,166,100001,-,-,-
3,401020053,EXCISAO E SUTURA DE LESAO NA PELE C/ PLASTICA ...,50,1.0,9,50,100002,-,-,-
4,401020088,EXERESE DE CISTO SACRO-COCCIGEO,594,0.7,9,415,100003,-,-,-
5,402010035,TIREOIDECTOMIA PARCIAL,22,0.0,9,0,100004,-,-,-
6,402010043,TIREOIDECTOMIA TOTAL,279,0.9,9,251,100005,-,-,-


#### Procedimento requer serviço/class

In [42]:
df_sigtap_s = df_sigtap[['COD_PROCEDIMENTO','EXIGE SERVIÇO','CO_SERVICO','CO_CLASSIFICACAO']] # Cria um novo dataframe com as colunas 'COD_PROCEDIMENTO','EXIGE SERVICO','CO_SERVICO'
df_sigtap_s.drop_duplicates(subset='COD_PROCEDIMENTO', keep='first', inplace=True) # Remove os valores duplicados da coluna 'COD_PROCEDIMENTO'
df_planilha_aba1['PROC_SERVICO'] = df_planilha_aba1['COD_PROCEDIMENTO'].map(df_sigtap_s.set_index('COD_PROCEDIMENTO')['EXIGE SERVIÇO']) # Adiciona uma nova coluna com a informação de serviço do procedimento
df_planilha_aba1.head() # Exibe as 5 primeiras linhas do arquivo

,COD_PROCEDIMENTO,DESC_PROCEDIMENTO,QUANT_FILA,PERC_REDUCAO,TEMPO_MESES,QUANT_REDUCAO,LINHA,M_DUPLICADO,QUANT_ZERO,PROC_HABILITACAO,PROC_SERVICO
2,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",166,1.0,9,166,100001,-,-,-,-
3,401020053,EXCISAO E SUTURA DE LESAO NA PELE C/ PLASTICA ...,50,1.0,9,50,100002,-,-,-,-
4,401020088,EXERESE DE CISTO SACRO-COCCIGEO,594,0.7,9,415,100003,-,-,-,-
5,402010035,TIREOIDECTOMIA PARCIAL,22,0.0,9,0,100004,-,-,-,-
6,402010043,TIREOIDECTOMIA TOTAL,279,0.9,9,251,100005,-,-,-,-


### Analise e tratamento da PLANILHA ABA 2

In [43]:
df_planilha_aba2 = pd.read_excel(df_planilha, sheet_name='Ident. CNES e Proced.', dtype=str) # Lê o arquivo excel
df_planilha_aba2.head() # Exibe as 5 primeiras linhas do arquivo

,PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - CNES,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Planilha protegida devido as fórmulas existent...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VRS 4,bahia
1,CNES,ESTABELECIMENTO,CÓDIGO DO PROCEDIMENTO PRINCIPAL NO SIGTAP,PROCEDIMENTO CIRÚRGICO (PRINCIPAL),COMPLEMENTO COM RECURSO FEDERAL DO PROCEDIMENT...,GESTÃO DO SERVIÇO,CODIGO DA NATUREZA JURÍDICA,NATUREZA JURÍDICA,POSSUI CONTRATO COM A SECRETARIA DE SAÚDE ?,Identificação do Nome do Estabelecimento que n...,SQ (CODIGO Interno
2,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,NaN,200001
3,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407040102,HERNIOPLASTIA INGUINAL / CRURAL (UNILATERAL),NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,NaN,200002
4,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407040129,HERNIOPLASTIA UMBILICAL,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,NaN,200003


In [44]:
# Tratamento dos dados da aba 'Ident. CNES e Proced.'
df_planilha_aba2.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - CNES':'CNES','Unnamed: 1':'ESTABELECIMENTO','Unnamed: 2':'COD_PROCEDIMENTO', 
                                 'Unnamed: 3':'DES_PROCEDIMENTO', 'Unnamed: 4':'COMPL_RECURSO_FEDERAL', 'Unnamed: 5':'GESTAO', 'Unnamed: 6':'COD_NATUREZA','Unnamed: 7':'NATUREZA',
                                 'Unnamed: 8':'POSSUI_CONTRATO','Unnamed: 9':'IDENTIFICACAO','Unnamed: 10':'LINHA'}, inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'
df_planilha_aba2.drop(0, inplace=True) # Remove a primeira linha do arquivo
df_planilha_aba2.drop(1, inplace=True) # Remove a primeira linha do arquivo
df_planilha_aba2.drop(df_planilha_aba2.tail(1).index,inplace=True) # Removendo a última linha do arquivo
df_planilha_aba2.dropna(subset=['CNES'], inplace=True) # Removendo linhas com valores nulos
df_planilha_aba2.drop('IDENTIFICACAO', axis=1, inplace=True) # Removendo coluna 'IDENTIFICACAO'
df_planilha_aba2['COD_PROCEDIMENTO'] = df_planilha_aba2['COD_PROCEDIMENTO'].astype(int) # Converte a coluna 'COD_PROCEDIMENTO' para string
df_cnes_habilitacao = df_cnes_habilitacao.rename(columns={"CO_CNES": "CNES"}) # Renomeia a coluna 'CO_CNES' para 'CNES'
df_planilha_aba2 = df_planilha_aba2.merge(df_cnes_habilitacao[["CNES", "CO_MUNICIPIO_GESTOR"]], on="CNES", how="left") # Adiciona a coluna 'CO_MUNICIPIO_GESTOR' ao dataframe

df_planilha_aba2.head() # Exibe as 5 primeiras linhas do arquivo

,CNES,ESTABELECIMENTO,COD_PROCEDIMENTO,DES_PROCEDIMENTO,COMPL_RECURSO_FEDERAL,GESTAO,COD_NATUREZA,NATUREZA,POSSUI_CONTRATO,LINHA,CO_MUNICIPIO_GESTOR
0,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840
1,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840
2,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840
3,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840
4,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840


In [45]:
df_planilha_aba2.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6473 entries, 0 to 6472
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   CNES                   6473 non-null   object
 1   ESTABELECIMENTO        6424 non-null   object
 2   COD_PROCEDIMENTO       6473 non-null   int32 
 3   DES_PROCEDIMENTO       6473 non-null   object
 4   COMPL_RECURSO_FEDERAL  0 non-null      object
 5   GESTAO                 6473 non-null   object
 6   COD_NATUREZA           6424 non-null   object
 7   NATUREZA               6424 non-null   object
 8   POSSUI_CONTRATO        5211 non-null   object
 9   LINHA                  6473 non-null   object
 10  CO_MUNICIPIO_GESTOR    6473 non-null   object
dtypes: int32(1), object(10)
memory usage: 581.6+ KB


#### Verificar se o CNES esta ATIVO:

In [46]:
df_cnes_habilitacao['CNES'] = df_cnes_habilitacao['CNES'].astype(str) # Converte a coluna 'CNES' para string
df_cnes_habilitacao2 = df_cnes_habilitacao.loc[df_cnes_habilitacao['CO_MOTIVO_DESAB'] > '0'] # Seleciona apenas os CNES habilitados
df_planilha_aba2['CNES_ATIVO'] = np.where(df_planilha_aba2['CNES'].isin(df_cnes_habilitacao2['CNES']), 'NÃO', '-') # Adiciona a coluna 'CNES_ATIVO' ao dataframe
df_planilha_aba2.head() # Exibe as 5 primeiras linhas do arquivo

,CNES,ESTABELECIMENTO,COD_PROCEDIMENTO,DES_PROCEDIMENTO,COMPL_RECURSO_FEDERAL,GESTAO,COD_NATUREZA,NATUREZA,POSSUI_CONTRATO,LINHA,CO_MUNICIPIO_GESTOR,CNES_ATIVO
0,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840,-
1,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840,-
2,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840,-
3,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840,-
4,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840,-


#### Verificar se existe prestador sem procedimento informado em fila (relação ABA1 E ABA2)

In [47]:
# Idenfica se o procedimento informado na aba anterior está na aba atual
df_planilha_aba2['PROC_INFORMADO'] = np.where(df_planilha_aba2['COD_PROCEDIMENTO'].isin(df_planilha_aba1['COD_PROCEDIMENTO']), '-','NÃO')     
df_planilha_aba2.head()                                

,CNES,ESTABELECIMENTO,COD_PROCEDIMENTO,DES_PROCEDIMENTO,COMPL_RECURSO_FEDERAL,GESTAO,COD_NATUREZA,NATUREZA,POSSUI_CONTRATO,LINHA,CO_MUNICIPIO_GESTOR,CNES_ATIVO,PROC_INFORMADO
0,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840,-,-
1,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840,-,-
2,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840,-,-
3,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840,-,-
4,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840,-,-


#### Verificar se existe procedimento sem prestador informado (relação ABA2 e ABA1)

In [48]:
# Verifica se o procedimento informado possui prestador informado
df_planilha_aba1_p = df_planilha_aba1.loc[df_planilha_aba1['QUANT_REDUCAO'] > 0] # Seleciona apenas os procedimentos com quantidade de redução maior que zero
df_planilha_aba1_p['POSSUI_PRESTADOR'] = np.where(df_planilha_aba1_p['COD_PROCEDIMENTO'].isin(df_planilha_aba2['COD_PROCEDIMENTO']),'SIM','NÃO')
df_planilha_aba1 = df_planilha_aba1.merge(df_planilha_aba1_p[['COD_PROCEDIMENTO','POSSUI_PRESTADOR']], on='COD_PROCEDIMENTO', how='left') # Adiciona a coluna 'POSSUI_PRESTADOR' ao dataframe
df_planilha_aba1.head()

,COD_PROCEDIMENTO,DESC_PROCEDIMENTO,QUANT_FILA,PERC_REDUCAO,TEMPO_MESES,QUANT_REDUCAO,LINHA,M_DUPLICADO,QUANT_ZERO,PROC_HABILITACAO,PROC_SERVICO,POSSUI_PRESTADOR
0,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",166,1.0,9,166,100001,-,-,-,-,SIM
1,401020053,EXCISAO E SUTURA DE LESAO NA PELE C/ PLASTICA ...,50,1.0,9,50,100002,-,-,-,-,SIM
2,401020088,EXERESE DE CISTO SACRO-COCCIGEO,594,0.7,9,415,100003,-,-,-,-,SIM
3,402010035,TIREOIDECTOMIA PARCIAL,22,0.0,9,0,100004,-,-,-,-,NaN
4,402010043,TIREOIDECTOMIA TOTAL,279,0.9,9,251,100005,-,-,-,-,SIM


#### Verificar se o procedimento informado é valido 

In [49]:
df_planilha_aba1['PROC_VALIDO'] = np.where(df_planilha_aba1['COD_PROCEDIMENTO'].isin(df_sigtap['COD_PROCEDIMENTO']), '-','NÃO')
df_planilha_aba1.head()

,COD_PROCEDIMENTO,DESC_PROCEDIMENTO,QUANT_FILA,PERC_REDUCAO,TEMPO_MESES,QUANT_REDUCAO,LINHA,M_DUPLICADO,QUANT_ZERO,PROC_HABILITACAO,PROC_SERVICO,POSSUI_PRESTADOR,PROC_VALIDO
0,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",166,1.0,9,166,100001,-,-,-,-,SIM,-
1,401020053,EXCISAO E SUTURA DE LESAO NA PELE C/ PLASTICA ...,50,1.0,9,50,100002,-,-,-,-,SIM,-
2,401020088,EXERESE DE CISTO SACRO-COCCIGEO,594,0.7,9,415,100003,-,-,-,-,SIM,-
3,402010035,TIREOIDECTOMIA PARCIAL,22,0.0,9,0,100004,-,-,-,-,NaN,-
4,402010043,TIREOIDECTOMIA TOTAL,279,0.9,9,251,100005,-,-,-,-,SIM,-


#### Verificar habilitação x CNES

In [50]:
df_planilha_aba2_h = df_planilha_aba2[['CNES','COD_PROCEDIMENTO']] # Cria um novo dataframe com as colunas 'CNES','COD_PROCEDIMENTO',
df_planilha_aba2_h.drop_duplicates(subset='COD_PROCEDIMENTO', keep='first', inplace=True) # Remove os valores duplicados da coluna 'COD_PROCEDIMENTO'
df_planilha_aba2_h['PROC_HABILITACAO'] = df_planilha_aba2_h['COD_PROCEDIMENTO'].map(df_sigtap_h.set_index('COD_PROCEDIMENTO')['EXIGE HABILITACAO']) # Adiciona uma nova coluna com a informação de habilitação do procedimento
df_planilha_aba2_h = df_planilha_aba2_h.merge(df_sigtap_h[['COD_PROCEDIMENTO','CO_HABILITACAO']], on='COD_PROCEDIMENTO', how='left') # Adiciona a coluna 'PROC_VALIDO' ao dataframe
df_planilha_aba2_h = df_planilha_aba2_h.merge(df_cnes_habilitacao[['CNES','CO_CODIGO_GRUPO']], on='CNES', how='left') # Adiciona a coluna 'PROC_VALIDO' ao dataframe
df_planilha_aba2_h.drop(df_planilha_aba2_h.loc[df_planilha_aba2_h['PROC_HABILITACAO'] == '-'].index, inplace=True) # Remove os procedimentos que não exigem habilitação
df_planilha_aba2_h['CNES_HABILITADO'] = np.where(df_planilha_aba2_h['CO_CODIGO_GRUPO'].isin(df_planilha_aba2_h['CO_HABILITACAO']), 'SIM','EXIGE_HAB') # Adiciona a coluna 'CNES_HABILITADO' ao dataframe
df_planilha_aba2_h.drop_duplicates(subset='COD_PROCEDIMENTO', keep='first', inplace=True) # Remove os valores duplicados da coluna 'CNES'
df_planilha_aba2 = df_planilha_aba2.merge(df_planilha_aba2_h[['CNES','COD_PROCEDIMENTO','CNES_HABILITADO']], on=['CNES','COD_PROCEDIMENTO'], how='left') # Adiciona a coluna 'CNES_HABILITADO' ao dataframe
df_planilha_aba2.drop_duplicates(subset='LINHA', keep='first', inplace=True) # Remove os valores duplicados da coluna 'CNES'
df_planilha_aba2.head()

,CNES,ESTABELECIMENTO,COD_PROCEDIMENTO,DES_PROCEDIMENTO,COMPL_RECURSO_FEDERAL,GESTAO,COD_NATUREZA,NATUREZA,POSSUI_CONTRATO,LINHA,CO_MUNICIPIO_GESTOR,CNES_ATIVO,PROC_INFORMADO,CNES_HABILITADO
0,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840,-,-,NaN
5,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407040102,HERNIOPLASTIA INGUINAL / CRURAL (UNILATERAL),NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200002,290840,-,-,NaN
10,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407040129,HERNIOPLASTIA UMBILICAL,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200003,290840,-,-,NaN
15,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,409060135,HISTERECTOMIA TOTAL,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200004,290840,-,-,NaN
20,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,409060119,HISTERECTOMIA C/ ANEXECTOMIA (UNI / BILATERAL),NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200005,290840,-,-,NaN


#### Verificar serviços/class x CNES

In [51]:
df_planilha_aba2_s = df_planilha_aba2[['CNES','COD_PROCEDIMENTO']] # Cria um novo dataframe com as colunas 'CNES','COD_PROCEDIMENTO',
df_planilha_aba2_s.drop_duplicates(subset='COD_PROCEDIMENTO', keep='first', inplace=True) # Remove os valores duplicados da coluna 'COD_PROCEDIMENTO'
df_planilha_aba2_s['EXIGE SERVIÇO'] = df_planilha_aba2_s['COD_PROCEDIMENTO'].map(df_sigtap_s.set_index('COD_PROCEDIMENTO')['EXIGE SERVIÇO']) # Adiciona uma nova coluna com a informação de habilitação do procedimento
df_planilha_aba2_s = df_planilha_aba2_s.merge(df_sigtap_s[['COD_PROCEDIMENTO','CO_SERVICO']], on='COD_PROCEDIMENTO', how='left') # Adiciona a coluna 'PROC_VALIDO' ao dataframe
df_cnes_servicos = df_cnes_servicos.rename(columns={"CO_CNES": "CNES"}) # Renomeia a coluna 'CO_CNES' para 'CNES'
df_planilha_aba2_s = df_planilha_aba2_s.merge(df_cnes_servicos[['CNES','CO_SERVICO']], on='CNES', how='left') # Adiciona a coluna 'PROC_VALIDO' ao dataframe
df_planilha_aba2_s.drop(df_planilha_aba2_s.loc[df_planilha_aba2_s['EXIGE SERVIÇO'] == '-'].index, inplace=True) # Remove os procedimentos que não exigem habilitação
df_planilha_aba2_s['CNES_SERVICO'] = np.where(df_planilha_aba2_s['CO_SERVICO_x'].isin(df_planilha_aba2_s['CO_SERVICO_y']), 'SIM','EXIGE_SERV') # Adiciona a coluna 'CNES_HABILITADO' ao dataframe
df_planilha_aba2_s.drop_duplicates(subset='COD_PROCEDIMENTO', keep='first', inplace=True) # Remove os valores duplicados da coluna 'CNES'
df_planilha_aba2 = df_planilha_aba2.merge(df_planilha_aba2_s[['CNES','COD_PROCEDIMENTO','CNES_SERVICO']], on=['CNES','COD_PROCEDIMENTO'], how='left') # Adiciona a coluna 'CNES_HABILITADO' ao dataframe
df_planilha_aba2.drop_duplicates(subset='LINHA', keep='first', inplace=True) # Remove os valores duplicados da coluna 'CNES'
df_planilha_aba2.head()

,CNES,ESTABELECIMENTO,COD_PROCEDIMENTO,DES_PROCEDIMENTO,COMPL_RECURSO_FEDERAL,GESTAO,COD_NATUREZA,NATUREZA,POSSUI_CONTRATO,LINHA,CO_MUNICIPIO_GESTOR,CNES_ATIVO,PROC_INFORMADO,CNES_HABILITADO,CNES_SERVICO
0,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200001,290840,-,-,NaN,NaN
1,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407040102,HERNIOPLASTIA INGUINAL / CRURAL (UNILATERAL),NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200002,290840,-,-,NaN,NaN
2,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407040129,HERNIOPLASTIA UMBILICAL,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200003,290840,-,-,NaN,NaN
3,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,409060135,HISTERECTOMIA TOTAL,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200004,290840,-,-,NaN,NaN
4,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,409060119,HISTERECTOMIA C/ ANEXECTOMIA (UNI / BILATERAL),NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,200005,290840,-,-,NaN,NaN


### Analise e tratamento da PLANILHA ABA 3

In [52]:
df_planilha_aba3 = pd.read_excel(df_planilha, sheet_name='Execução') # Lê o arquivo excel
df_planilha_aba3.head(10) # Exibe as 5 primeiras linhas do arquivo

,Distribuição e Cronograma da Execução do Recurso Financeiro,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,SQ (CODIGO Interno
0,Selecione a UF (Utilize a barra de rolagem) >,bahia,Portaria GM nº 90/2023,42149212.05,< Valor Publicado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,399999
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,Planilha Protegida devido as fórmulas existent...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VRS 4,NaN
3,NaN,NaN,Saldo Disponível,0,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Total distribuído >,42149212.05,Cronograma de Execução (%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CODIGO GESTOR,Gestão do Recurso,DESCRIÇÃO DO GESTOR,VALOR,março,abril,maio,junho,julho,agosto,setembro,outubro,novembro,dezembro,% TOTAL,SQ (CODIGO Interno
6,290000,ESTADUAL,Bahia,21182803.42,NaN,0.1,0.1,0.1,0.15,0.1,0.1,0.15,0.1,0.1,1,30001
7,290570,MUNICIPAL,CAMACARI,759658.17,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1.0,30002
8,290600,MUNICIPAL,CAMPO FORMOSO,176284.29,NaN,NaN,0.2,0.05,0.15,0.2,0.2,0.2,NaN,NaN,1,30003
9,291005,MUNICIPAL,DIAS D'AVILA,205645.35,NaN,NaN,0.15,0.15,0.15,0.15,0.2,0.1,0.1,NaN,1,30004


In [53]:
valor_portaria = df_planilha_aba3.iloc[0,3] # Armazena o valor da portaria
df_planilha_aba3.drop(df_planilha_aba3.index[0:6], inplace=True) # Remove as 5 primeiras linhas do arquivo
df_planilha_aba3.rename(columns={'Distribuição e Cronograma da Execução do Recurso Financeiro':'CODIGO GESTOR','Unnamed: 1':'GESTÃO','Unnamed: 2':'DESC_GESTOR','Unnamed: 3':'VALOR','Unnamed: 4':'MARÇO',
                                 'Unnamed: 5':'ABRIL','Unnamed: 6':'MAIO','Unnamed: 7':'JUNHO','Unnamed: 8':'JULHO','Unnamed: 9':'AGOSTO','Unnamed: 10':'SETEMBRO','Unnamed: 11':'OUTUBRO',
                                 'Unnamed: 12':'NOVEMBRO','Unnamed: 13':'DEZEMBRO','Unnamed: 14':'TOTAL_%','SQ (CODIGO Interno':'LINHA'}, inplace=True)
df_planilha_aba3.drop(df_planilha_aba3.tail(1).index,inplace=True) # Removendo a última linha do arquivo
df_planilha_aba3.dropna(subset=['CODIGO GESTOR'], inplace=True) # Removendo linhas com valores nulos
valor_total = df_planilha_aba3['VALOR'].sum() # Soma o valor total da coluna 'VALOR'
df_planilha_aba3['VALOR'] = df_planilha_aba3['VALOR'].astype(float) # Converte a coluna 'VALOR' para float
df_planilha_aba3['VALOR_R'] = df_planilha_aba3['VALOR'].apply(lambda x: 'R$ ' + format(x, ',.2f').replace('.', '#').replace(',', '.').replace('#', ',')) # Formata a coluna 'VALOR' para moeda

df_planilha_aba3.head() # Exibe as 5 primeiras linhas do arquivo

,CODIGO GESTOR,GESTÃO,DESC_GESTOR,VALOR,MARÇO,ABRIL,MAIO,JUNHO,JULHO,AGOSTO,SETEMBRO,OUTUBRO,NOVEMBRO,DEZEMBRO,TOTAL_%,LINHA,VALOR_R
6,290000,ESTADUAL,Bahia,21182803.42,NaN,0.1,0.1,0.1,0.15,0.1,0.1,0.15,0.1,0.1,1,30001,"R$ 21.182.803,42"
7,290570,MUNICIPAL,CAMACARI,759658.17,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1.0,30002,"R$ 759.658,17"
8,290600,MUNICIPAL,CAMPO FORMOSO,176284.29,NaN,NaN,0.2,0.05,0.15,0.2,0.2,0.2,NaN,NaN,1,30003,"R$ 176.284,29"
9,291005,MUNICIPAL,DIAS D'AVILA,205645.35,NaN,NaN,0.15,0.15,0.15,0.15,0.2,0.1,0.1,NaN,1,30004,"R$ 205.645,35"
10,291480,MUNICIPAL,ITABUNA,526054.58,0.1,0.1,0.1,0.1,0.11,0.1,0.1,0.1,0.1,0.09,1.0,30005,"R$ 526.054,58"


In [54]:
df_planilha_aba3['CODIGO GESTOR'] = df_planilha_aba3['CODIGO GESTOR'].astype(int) # Converte a coluna 'MUNIC_PRESTADOR' para string
df_planilha_aba2['CO_MUNICIPIO_GESTOR'] = df_planilha_aba2['CO_MUNICIPIO_GESTOR'].astype(int) # Converte a coluna 'CO_MUNICIPIO_GESTOR' para string
df_planilha_aba3['MUNIC_PRESTADOR'] = np.where(df_planilha_aba3['CODIGO GESTOR'].isin(df_planilha_aba2['CO_MUNICIPIO_GESTOR']), 'SIM','NÃO')
df_planilha_aba3_m = df_planilha_aba3[df_planilha_aba3['GESTÃO'] == 'MUNICIPAL'] # Filtra os dados da aba 3 que possuem gestão municipal
df_teto['Código IBGE'] = df_teto['Código IBGE'].astype(int) # Converte a coluna 'Código IBGE' para string
df_planilha_aba3 = pd.merge(df_planilha_aba3, df_teto, left_on='CODIGO GESTOR', right_on='Código IBGE', how='left') # Realiza o merge das duas planilhas

df_planilha_aba3.drop(['Código IBGE', 'Código Gestão', 'Descrição Gestão'], axis=1, inplace=True) # Remove a coluna 'Código IBGE'
df_planilha_aba3.rename(columns={'Descrição Gestão':'DESC_GESTOR','Estado / Município':'DESC_GESTOR_C','Teto Financeiro MAC - Valores Anuais (R$)':'TETO_FINANC_ANO'}, inplace=True) # Renomeia a coluna 'Descrição Gestão'
df_planilha_aba3['TETO_FINANC_ANO'] = df_planilha_aba3['TETO_FINANC_ANO'].astype(float) # Converte a coluna 'TETO_FINANC_ANO' para float
df_planilha_aba3['TETO_FINANC_MES'] = df_planilha_aba3['TETO_FINANC_ANO'] / 12 # Calcula o teto financeiro mensal

df_planilha_aba3.head()


,CODIGO GESTOR,GESTÃO,DESC_GESTOR,VALOR,MARÇO,ABRIL,MAIO,JUNHO,JULHO,AGOSTO,SETEMBRO,OUTUBRO,NOVEMBRO,DEZEMBRO,TOTAL_%,LINHA,VALOR_R,MUNIC_PRESTADOR,Sigla UF,DESC_GESTOR_C,TETO_FINANC_ANO,TETO_FINANC_MES
0,290000,ESTADUAL,Bahia,21182803.42,NaN,0.1,0.1,0.1,0.15,0.1,0.1,0.15,0.1,0.1,1,30001,"R$ 21.182.803,42",NÃO,BA,BAHIA,1.452572e+09,1.210477e+08
1,290570,MUNICIPAL,CAMACARI,759658.17,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1.0,30002,"R$ 759.658,17",SIM,BA,CAMAÇARI,2.023838e+07,1.686532e+06
2,290600,MUNICIPAL,CAMPO FORMOSO,176284.29,NaN,NaN,0.2,0.05,0.15,0.2,0.2,0.2,NaN,NaN,1,30003,"R$ 176.284,29",SIM,BA,CAMPO FORMOSO,4.231395e+06,3.526162e+05
3,291005,MUNICIPAL,DIAS D'AVILA,205645.35,NaN,NaN,0.15,0.15,0.15,0.15,0.2,0.1,0.1,NaN,1,30004,"R$ 205.645,35",SIM,BA,DIAS D'ÁVILA,8.533689e+06,7.111407e+05
4,291480,MUNICIPAL,ITABUNA,526054.58,0.1,0.1,0.1,0.1,0.11,0.1,0.1,0.1,0.1,0.09,1.0,30005,"R$ 526.054,58",SIM,BA,ITABUNA,1.114601e+08,9.288341e+06


In [55]:
df_planilha_aba3.head() # Exibe as informações do dataframe

,CODIGO GESTOR,GESTÃO,DESC_GESTOR,VALOR,MARÇO,ABRIL,MAIO,JUNHO,JULHO,AGOSTO,SETEMBRO,OUTUBRO,NOVEMBRO,DEZEMBRO,TOTAL_%,LINHA,VALOR_R,MUNIC_PRESTADOR,Sigla UF,DESC_GESTOR_C,TETO_FINANC_ANO,TETO_FINANC_MES
0,290000,ESTADUAL,Bahia,21182803.42,NaN,0.1,0.1,0.1,0.15,0.1,0.1,0.15,0.1,0.1,1,30001,"R$ 21.182.803,42",NÃO,BA,BAHIA,1.452572e+09,1.210477e+08
1,290570,MUNICIPAL,CAMACARI,759658.17,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1.0,30002,"R$ 759.658,17",SIM,BA,CAMAÇARI,2.023838e+07,1.686532e+06
2,290600,MUNICIPAL,CAMPO FORMOSO,176284.29,NaN,NaN,0.2,0.05,0.15,0.2,0.2,0.2,NaN,NaN,1,30003,"R$ 176.284,29",SIM,BA,CAMPO FORMOSO,4.231395e+06,3.526162e+05
3,291005,MUNICIPAL,DIAS D'AVILA,205645.35,NaN,NaN,0.15,0.15,0.15,0.15,0.2,0.1,0.1,NaN,1,30004,"R$ 205.645,35",SIM,BA,DIAS D'ÁVILA,8.533689e+06,7.111407e+05
4,291480,MUNICIPAL,ITABUNA,526054.58,0.1,0.1,0.1,0.1,0.11,0.1,0.1,0.1,0.1,0.09,1.0,30005,"R$ 526.054,58",SIM,BA,ITABUNA,1.114601e+08,9.288341e+06


In [56]:
diferenca = valor_portaria - valor_total # Calcula a diferença entre o valor da portaria e o valor total
diferenca = 'R$ ' + format(diferenca, ',.2f').replace('.', '#').replace(',', '.').replace('#', ',') # Formata a diferença para moeda
print(diferenca)

R$ 0,00


# SALVANDO OS RESULTADOS    

In [57]:
df_planilha = os.path.splitext(os.path.basename(df_planilha))[0] # Pega o nome do arquivo sem a extensão
file_nome = df_planilha.split('/')[-1] # Pega o nome do arquivo
writer = pd.ExcelWriter(f'PLANILHA/{file_nome}_resultado.xlsx', engine='xlsxwriter') # Cria um arquivo excel

df_planilha_aba1.to_excel(writer, sheet_name='Aba 1', index=False)
df_planilha_aba2.to_excel(writer, sheet_name='Aba 2', index=False)
df_planilha_aba3.to_excel(writer, sheet_name='Aba 3', index=False)

writer.save()


# RELATORIO FINAL (ANALISE)

In [60]:
# Verificação de procedimentos duplicados
if df_planilha_aba1['M_DUPLICADO'].str.contains('SIM').any():
    print(f"{colors.ERRO}[ERRO] - ABA 1 - Existem procedimentos na Fila, duplicado;")
    procedimento_duplicado = True
else:
    print(f"{colors.OK}[OK] - ABA 1 - Não existem procedimentos duplicados;")
    procedimento_duplicado = False


# VerificaçãO de procedimentos com quantidade menor que 1
if df_planilha_aba1['QUANT_ZERO'].str.contains('ERRO_QUANT').any():   
    print(f"{colors.ERRO}[ERRO] - ABA 1 - Existem procedimentos na Fila, cuja quantidade de cirurgia ficou inferior a 1 procedimento (PACIENTE);")
    quant_menor_1 = True
else:
    print(f"{colors.OK}[OK] - ABA 1 - Não existem procedimentos com quantidade menor que 1;")
    quant_menor_1 = False


# Verificação de procedimentos inválidos
if df_planilha_aba1['PROC_VALIDO'].str.contains('NÃO').any():
    print(f"{colors.ERRO}[ERRO] - ABA 1 - Existem procedimentos na Fila, que não são válidos;")
    procedimento_invalido = True
else:
    print(f"{colors.OK}[OK] - ABA 1 - Não existem procedimentos inválidos;")
    procedimento_invalido = False    


# Verificação de procedimentos sem prestador
if df_planilha_aba1['POSSUI_PRESTADOR'].str.contains('NÃO').any():
    print(f"{colors.ERRO}[ERRO] - ABA 1/2 - Existem procedimentos na Fila, cuja não existe prestador;")
    sem_prestador = True
else:
    print(f"{colors.OK}[OK] - ABA 1/2 - Não existem procedimentos sem prestador;")
    sem_prestador = False


# Verificação de CNES ativo
if df_planilha_aba2['CNES_ATIVO'].str.contains('NÃO').any():
    print(f"{colors.ERRO}[ERRO] - ABA 2 - Existem CNES inativos;")
    cnes_inativo = True
else:
    print(f"{colors.OK}[OK] - ABA 2 - Não existem CNES inativos;")
    cnes_inativo = False

# Verificação de CNES habilitado
if df_planilha_aba2['CNES_HABILITADO'].str.contains('EXIGE_HAB').any():
    print(f"{colors.NORMAL}[ALERTA] - ABA 2 - Existem CNES não habilitados;")
    cnes_nao_habilitado = True
else:
    print(f"{colors.OK}[OK] - ABA 2 - Não existem CNES não habilitados;")
    cnes_nao_habilitado = False

# Verificação de CNES serviço ativo
if df_planilha_aba2['CNES_SERVICO'].str.contains('EXIGE_SERV').any():
    print(f"{colors.ERRO}[ERRO] - ABA 2 - Existem CNES não habilitados;")
    cnes_nao_habilitado = True
else:
    print(f"{colors.OK}[OK] - ABA 2 - Não existem CNES não serviço/class;")
    cnes_nao_habilitado = False


# Verificar valor de portaria
if valor_total == valor_portaria:
    print(f"{colors.OK}[OK] - ABA 3 - Valor programado igual ao valor alocado na Portaria 90;")
    valor_portaria_correto = True
else:   
    print(f"{colors.ERRO}[ERRO] - ABA 3 - Valor programado diferente do valor alocado na Portaria 90; {diferenca}")
    valor_portaria_correto = False
    

# Verificar se existe municipio com repasse, mas não existe CNES relacionado ao municipio
if df_planilha_aba3_m['MUNIC_PRESTADOR'].str.contains('NÃO').any():   
    print(f"{colors.ERRO}[ERRO] - ABA 3 - Existem municipio com repasse, mas não existe CNES relacionado ao municipio;")
    
    sem_prestador = True
else:   
    print(f"{colors.OK}[OK] - ABA 3 - Relação de municipios diacordo com o CNES;")
    sem_prestador = False

# Arquivo gerado com os resultados
print(f"{colors.OK}[OK] - Arquivo: '{file_nome}- resultado.xlsx' gerado com sucesso!")






[OK] - ABA 1 - Não existem procedimentos duplicados;
[OK] - ABA 1 - Não existem procedimentos com quantidade menor que 1;
[OK] - ABA 1 - Não existem procedimentos inválidos;
[OK] - ABA 1/2 - Não existem procedimentos sem prestador;
[OK] - ABA 2 - Não existem CNES inativos;
[ALERTA] - ABA 2 - Existem CNES não habilitados;
[OK] - ABA 2 - Não existem CNES não serviço/class;
[OK] - ABA 3 - Valor programado igual ao valor alocado na Portaria 90;
[OK] - ABA 3 - Relação de municipios diacordo com o CNES;
[OK] - Arquivo: 'plano-atendimento-perf-cir-eletiva-vrs-4a SAIPS FINAL- resultado.xlsx' gerado com sucesso!


In [59]:
tempo_final = time.time()
tempo_total = (float(tempo_final - tempo_inicial)/60)
print(f"Tempo total de execução: {tempo_total:,.1f} minutos" )

Tempo total de execução: 0.2 minutos
